In [1]:
import or_suite
import numpy as np
import gym

CONFIG = or_suite.envs.env_configs.rideshare_graph_default_config

In [2]:
# making an instance of the environment
env = gym.make('Rideshare-v0', config=CONFIG)

In [3]:
print(env.state)
print(env.action_space)
print(env.observation_space)

[2 1 4 3 0 0]
Discrete(4)
MultiDiscrete([10 10 10 10  4  4])


In [4]:
# testing the step function
print(env.state)
newState, reward,  done, info = env.step(0)
print(newState)
print(reward)
print(done)

[2 1 4 3 0 0]
0.5 0
decline service
[2 1 4 3 0 0]
-10000
False


In [5]:
newState, reward,  done, info = env.step(1)
print(newState)
print(reward)
print(done)

0.2689414213699951 1
accept service
[3 0 4 3 0 0]
-1.0
False


In [6]:
newState, reward,  done, info = env.step(0)
print(newState)
print(reward)
print(done)

0.5 1
accept service
[3 0 4 3 1 0]
-0.0
False


In [7]:
newState, reward,  done, info = env.step(0)
print(newState)
print(reward)
print(done)

0.2689414213699951 1
accept service
[3 0 4 3 3 0]
-1.0
False


### Step 1: Import Required Packages

The main package for ORSuite is contained in `or_suite`.  However, some additional packages may be required for specific environments / algorithms.  Here, we include `stable baselines`, a package containing implementation for state of the art deep RL algorithms, and `matploblib` for the plotting.

In [8]:
import or_suite
import gym
import matplotlib.pyplot as plt
from stable_baselines3.common.monitor import Monitor
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
import numpy as np

### Step 2: Pick problem parameters for the environment

Here we use the ambulance metric environment as outlined in `or_suite/envs/ambulance/ambulance_metric.py`.  The package has default specifications for all of the environments in the file `or_suite/envs/env_configs.py`, and so we use one the default for the ambulance problem in a metric space.

In addition, we need to specify the number of episodes for learning, and the number of iterations (in order to plot average results with confidence intervals).

In [9]:
DEFAULT_CONFIG = or_suite.envs.env_configs.rideshare_graph_default_config
epLen = DEFAULT_CONFIG['epLen']
nEps = 200
numIters = 5

### Step 3: Pick simulation parameters

Next we need to specify parameters for the simulation.  This includes setting a seed, the frequency to record the metrics, directory path for saving the data files, a deBug mode which prints the trajectory, etc.

In [10]:
DEFAULT_SETTINGS = {'seed': 1, 
                    'recFreq': 1, 
                    'dirPath': './data/rideshare/', 
                    'deBug': False, 
                    'nEps': nEps, 
                    'numIters': numIters,
                    'render': False,
                    'saveTrajectory': True, 
                    'epLen' : 5}

rideshare_env = gym.make('Rideshare-v0', config=DEFAULT_CONFIG)
mon_env = Monitor(rideshare_env)

### Step 4: Pick list of algorithms

We have several heuristics implemented for each of the environments defined, in addition to a `random` policy, and some `RL discretization based` algorithms.  Here we pick a couple of the heuristics, and a PPO algorithm implemented from `stable baselines` just to test.

In [11]:
agents = {'Random': or_suite.agents.rl.random.randomAgent(),
          'SB PPO': PPO(MlpPolicy, mon_env, gamma=1, verbose=0, n_steps=epLen),
          }

/home/sean/anaconda3/envs/ORSuite/lib/python3.8/site-packages/stable_baselines3/ppo/ppo.py:131: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 5`, after every 0 untruncated mini-batches, there will be a truncated mini-batch of size 5
We recommend using a `batch_size` that is a multiple of `n_steps * n_envs`.
Info: (n_steps=5 and n_envs=1)
  warnings.warn(


### Step 5: Run simulations

In [12]:
#or_suite.utils.run_single_algo(mon_env, agents['Random'], DEFAULT_SETTINGS)

In [13]:
for agent in agents:
    print(agent)
    DEFAULT_SETTINGS['dirPath'] = './data/vaccine_metric_test_'+str(agent)+'/'
    if agent == 'SB PPO':
        or_suite.utils.run_single_sb_algo(mon_env, agents[agent], DEFAULT_SETTINGS)
    else:
        or_suite.utils.run_single_algo(mon_env, agents[agent], DEFAULT_SETTINGS)

Random
**************************************************
Running experiment
**************************************************
0.5 1
accept service
0.2689414213699951 0
decline service
0.5 0
decline service
0.2689414213699951 0
decline service
0.5 1
accept service
0.2689414213699951 1
accept service
0.2689414213699951 0
decline service
0.5 1
accept service
0.2689414213699951 1
accept service
0.2689414213699951 0
decline service
0.5 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.5 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 1
accept service
0.2689414213699951 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.26894142

0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.5 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.5 1
accept service
0.2689414213699951 1
accept service
0.5 1
accept service
0.2689414213699951 0
decline service
0.5 1
accept service
0.2689414213699951 1
accept service
0.2689414213699951 1
accept service
0.5 0
decline service
0.2689414213699951 1
accept service
0.5 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.5 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 1
accept service
0.5 0
decline service
0.2689414213699951 

0.2689414213699951 0
decline service
0.2689414213699951 1
accept service
0.2689414213699951 1
accept service
0.2689414213699951 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 1
accept service
0.2689414213699951 0
decline service
0.5 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 1
accept service
0.5 0
decline service
0.2689414213699951 0
decline service
0.5 1
accept service
0.2689414213699951 0
decline service
0.5 1
accept service
0.2689414213699951 0
decline service
0.5 0
decline service
0.2689414213699951 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 1
accept service
0.5 1
accept service
0.2689414213699951 0
decline

0.5 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 1
accept service
0.5 0
decline service
0.2689414213699951 0
decline service
0.5 1
accept service
0.5 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 1
accept service
0.5 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 1
accept service
0.2689414213699951 1
accept service
0.5 0
decline service
0.2689414213699951 0
declin

0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.5 0
decline service
0.2689414213699951 0
decline service
0.5 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.5 1
accept service
0.5 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.5 0
decline service
0.2689414213699951 0
decline service
0.5 0
decline service
0.2689414213699951 1
accept service
0.2689414213699951 1
accept service
0.2689414213699951 1
accept service
0.2689414213699951 1
accept service
0.5 0
decline service
0.2689414213699951 1
accept service
0.5 1
accept service
0.2689414213699951 0
decline service
0.5 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 1
accept service

...

0.2689414213699951 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.5 0
decline service
0.5 0
decline service
0.5 0
decline service
0.2689414213699951 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 1
accept service
0.2689414213699951 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 1
accept service
0.5 1
accept service
0.2689414213699951 0
decline service
0.5 1
accept service
0.2689414213699951 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.5 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.5 0
decline service
0.2689414213699951 0
decline service
0.5 1
accept service
0.2689414213699951 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.5 1
accept service
0.5 1
accept service
0.2689414213699951 0
decline service
0.2689414213699

0.2689414213699951 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.5 1
accept service
0.5 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.5 0
decline service
0.5 0
decline service
0.2689414213699951 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 1
accept service
0.5 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 1
accept service
0.5 0
decline service
0.5 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.

0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.5 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 1
accept service
0.5 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 1
accept service
0.2689414213699951 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.5 1
accept service
0.2689414213699951 0
decline service
0.5 0
decline service
0.2689414213699951 0
decline service
0.5 1
accept service
0.2689414213699951 0
decline service
0.5 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.5 0
decline service
0.5 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 1
accept service
0.2689414213699951 1

0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.2689414213699951 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 1
accept service
0.2689414213699951 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.5 0
decline service
0.5 1
accept service
0.2689414213699951 1
accept service
0.5 1
accept service
0.2689414213699951 0
decline service
0.5 1
accept service
0.2689414213699951 0
decline service
0.2689414213699951 0
decline service
0.5 1
accept service
0.2689414213699951 1
accept service
0.5 0
decline service
0.2689414213699951 1
accept service
0.2689414213699951 0
decline service
0.5 0
decline service
0.2689414213699951 0
decline service
0.26894

ValueError: arrays must all be same length

In [ ]:
from stable_baselines3.common.env_checker import check_env

In [ ]:
check_env(rideshare_env, skip_render_check=True)